# Objetivo

El objetivo de esta libreta es explorar el procesamiento y análisis de los datos con PySpark, así como explorar los RDDs y dataframes.

Para realizar los ejemplos de este notebook haremos uso de una base de datos que cuenta con información de los jugadores del juego [FIFA20](https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_20.csv), estos datos fueron tomados del sitio de kaggle donde podrán encontrar una descripción detallada de cada uno de los campos.

## ¿Qué es PySpark?
PySpark es una herramienta que nos deja usar Spark encima de Python. Permitiendonos combinar el proceso de datos distribuidos de Spark con la simplicidad de Python para el análisis de conjuntos masivos de datos (big data).

In [142]:
# !pip install pyspark

In [143]:
#Carga de la biblioteca
import pyspark
# Carga funciones extra
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()

### Operaciones con RDDs

La forma más sencilla de trabajar con PySpark es usando RDDs (Resilient Distributed Dataset).

Los RDDs son una colección de datos, con las características que son tolerantes a fallos y pueden trabajar de manera paralela.

In [144]:
#Carga de los datos
#Mi primer RDD (cargamos los datos de un CSV hacia un RDD)
datos_crudos = spark.sparkContext.textFile('players_20.csv')

In [145]:
datos_crudos

players_20.csv MapPartitionsRDD[350] at textFile at NativeMethodAccessorImpl.java:0

In [146]:
for renglon in datos_crudos.take(2):
  print(renglon)

sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defe

Una de las desventajas de los RDDs es que solo proveen a los datos de la estructura minima para poder trabajar con ellos. Como podemos ver, los elementos del RDD son solo los renglones del archivo CSV sin formato.

In [147]:
# Guarda el primer elemento
columnas = datos_crudos.take(1)[0]
# Guarda en todos los elementos de datos_crudos menos el primero correspondiente a las columnas
datos_crudos = datos_crudos.filter(lambda renglon : renglon != columnas)

In [148]:
datos_crudos

PythonRDD[353] at RDD at PythonRDD.scala:53

Ahora podemos darle un poco más de estructura a nuestros datos. Primero separemos los campos de cada renglon, sabiendo que estan separados por comas, y quedandonos con solo un subconjunto de dichos campos, digamos... su nombre, edad, estatura, peso, nacionalidad, equipo y su ranking. Para esto definamos la siguiente función:

In [149]:
def separar_campos(renglon):
    renglon_separado = renglon.split(',')
    # Extrae los campos que queremos
    nombre = str(renglon_separado[2])
    edad = int(renglon_separado[4])
    estatura = int(renglon_separado[6])
    peso = int(renglon_separado[7])
    nacionalidad = str(renglon_separado[8])
    equipo = str(renglon_separado[9])
    rank = int(renglon_separado[10])
    return (nombre, edad, estatura, peso, nacionalidad, equipo, rank)

In [150]:
#aplicamos la función a los datos

rdd_fifa = datos_crudos.map(separar_campos)
# Imprimimos una muestra de los datos
for renglon in rdd_fifa.take(5):
  print(renglon)

('L. Messi', 32, 170, 72, 'Argentina', 'FC Barcelona', 94)
('Cristiano Ronaldo', 34, 187, 83, 'Portugal', 'Juventus', 93)
('Neymar Jr', 27, 175, 68, 'Brazil', 'Paris Saint-Germain', 92)
('J. Oblak', 26, 188, 87, 'Slovenia', 'Atlético Madrid', 91)
('E. Hazard', 28, 175, 74, 'Belgium', 'Real Madrid', 91)


Existen dos operaciones básicas que pueden ser aplicadas a los RDD:

- Acciones: Son operaciones que accionan la realización de cálculos y regresan valores explícitos del RDD. Ejemplo: take()

- Transformaciones: Son operaciones que devuelven otro RDD. Este tipo de operaciones no se ejecutan como tal sino hasta que es llamada una acción. Por esta razón se dice que las transformaciones son de evaluacion peresoza (lazy evaluation). Ejemplos: map(), reduceByKey()

En el caso de la celda anterior, map() es una transformación que aplica la función dada a los elementos del RDD y take() es la acción que nos devuelve los  primeros elementos.

##### Ejemplo 1: ¿Cuántos jugadores existen por país?

Para lograr esto primero necesitamos convertir nuestros datos en pares (key, value), donde la key será el pais y el value nos indicará que ese registro pertenece a un jugador de dicho país.

In [151]:
# Nos quedamos solo con la entrada corresponiente al país
rdd_nacionalidades = rdd_fifa.map(lambda renglon: (renglon[4], 1))

In [152]:
for i in rdd_nacionalidades.take(5):
    print(i)

('Argentina', 1)
('Portugal', 1)
('Brazil', 1)
('Slovenia', 1)
('Belgium', 1)


In [153]:
#Usando los países como llaves y la transformación llamada reduceByKey, sumaremos todos los valores para cada país.
rdd_nacionalidades = rdd_nacionalidades.reduceByKey(lambda val_1 , val_2: val_1 + val_2)

In [154]:
#Para visualizar los resultados, usaremos la acción collect(). Esta acción devuelve todo el contenido del RDD.

resultados = rdd_nacionalidades.collect()

for renglon in resultados[:10]:
  print(renglon)

('Germany', 1216)
('Egypt', 30)
('France', 984)
('Italy', 732)
('Gabon', 16)
('Costa Rica', 30)
('Slovakia', 54)
('Bosnia Herzegovina', 66)
('Serbia', 139)
('Scotland', 277)


In [155]:
#Importante: Esta acción debe usarse con cuidado, en casos donde el RDD contenga una gran cantidad de registros, cargar todo el contenido del RDD puede dejar al sistema sin memoria.

##### Ejemplo 2: Ordenar a los paises por la cantidad de jugadores.

En la siguiente celda imprimimos los 10 países con más jugadores registrados en el juego de FIFA20

In [156]:
# Pedimos que ordene por total de jugadores
rdd_nacionalidades = rdd_nacionalidades.sortBy(lambda renglon: renglon[1], ascending = False)

for renglon in rdd_nacionalidades.take(5):
  print(renglon)

('England', 1667)
('Germany', 1216)
('Spain', 1035)
('France', 984)
('Argentina', 886)


Aunque los RDDs nos proveen de la capacidad de manipular y hacer calculos con datos distribuidos, éstos carecen de las funcionalidades y la estructura que tienen herramientas de un poco más alto nivel, como los DataFrame de Pandas, para el manejo de datos.

En PySpark existe un tipo de objetos, igual llamados DataFrames, los cuales nos ayudan a dotar a nuestros datos con más funcionalidades y estructura, en comparación a los RDDs, al mismo tiempo que seguir trabajando de forma distribuida con ellos.

## Operaciones sobre DataFrames

A diferencia de los RDDs, los DataFrames nos permiten estructurar a una collección de datos distribuidos en forma de tabla (con columnas y filas).

In [157]:
#Mi primer Dataframe en PySpark
df_fifa = spark.read.csv('players_20.csv', header = True, inferSchema = True)

In [158]:
df_fifa

DataFrame[sofifa_id: int, player_url: string, short_name: string, long_name: string, age: int, dob: date, height_cm: int, weight_kg: int, nationality: string, club: string, overall: int, potential: int, value_eur: int, wage_eur: int, player_positions: string, preferred_foot: string, international_reputation: int, weak_foot: int, skill_moves: int, work_rate: string, body_type: string, real_face: string, release_clause_eur: int, player_tags: string, team_position: string, team_jersey_number: int, loaned_from: string, joined: date, contract_valid_until: int, nation_position: string, nation_jersey_number: int, pace: int, shooting: int, passing: int, dribbling: int, defending: int, physic: int, gk_diving: int, gk_handling: int, gk_kicking: int, gk_reflexes: int, gk_speed: int, gk_positioning: int, player_traits: string, attacking_crossing: int, attacking_finishing: int, attacking_heading_accuracy: int, attacking_short_passing: int, attacking_volleys: int, skill_dribbling: int, skill_curve: 

In [159]:
#Semejante a pandas, podemos seleccionar un conjunto de columnas
df_fifa = df_fifa.select('short_name','age', 'height_cm', 'weight_kg', 'nationality', 'club', 'value_eur', 'preferred_foot')

# dada la estructrua que el DataFrame da a los datos, podemos hacer referencia a las columnas por su nombre en el archivo CSV.

### Visualizando datos

In [160]:
df_fifa.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



In [161]:
#Si el contenido del DataFrame es lo suficientemente pequeño podemos transformar su contenido a un DataFrame de Pandas.
df_pequeño = df_fifa.limit(10)
df_pandas = df_pequeño.toPandas()
df_pandas

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right
5,K. De Bruyne,28,181,70,Belgium,Manchester City,90000000,Right
6,M. ter Stegen,27,187,85,Germany,FC Barcelona,67500000,Right
7,V. van Dijk,27,193,92,Netherlands,Liverpool,78000000,Right
8,L. Modrić,33,172,66,Croatia,Real Madrid,45000000,Right
9,M. Salah,27,175,71,Egypt,Liverpool,80500000,Left


In [162]:
#si quiero visualizar solo las primeras 5 filas, entonces:
df_pandas.head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


In [163]:
#Otra forma de hace lo mismo es:
df_fifa.limit(10).toPandas().head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


In [164]:
#Importante: en SPark es común encontrar la siguiente sintáxis. La cual se usa únicamente para dar mayor claridad el código.
df_fifa.limit(10) \
  .toPandas() \
  .head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


In [165]:
#Para poder revisar la organización de nuestro DataFrame y el data type de cada campo podemos usarlo siguiente

df_fifa.printSchema()

root
 |-- short_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)



#### Renombrar columnas

In [166]:
df_fifa.withColumnRenamed('value_eur','valor_euros').show(5) #esto no afectará al dataframe original

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



In [167]:
#Para tener resultados permanentes, debemos guardar la salida en otro dataframe.
df_ejemplo = df_fifa.withColumnRenamed('value_eur','valor_euros')
df_ejemplo.show(5)

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



#### Eliminar columnas

In [168]:
df_fifa.drop('weight_kg').show(5)

+-----------------+---+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|    Belgium|        Real Madrid| 90000000|         Right|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### Ordenar columnas

In [169]:
#Podemos ordenar a los jugadores por su valor en euros
df_fifa.orderBy('value_eur', ascending = False).show(5)

+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|  short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|   Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|    L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|   K. Mbappé| 20|      178|       73|     France|Paris Saint-Germain| 93500000|         Right|
|   E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
|K. De Bruyne| 28|      181|       70|    Belgium|    Manchester City| 90000000|         Right|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### Filtrar datos

In [170]:
#con la funcion where() podemos hacer un filtrado de nuestros datos
df_fifa.where(df_fifa.age > 35).show(5)

+---------------+---+---------+---------+-----------+---------+---------+--------------+
|     short_name|age|height_cm|weight_kg|nationality|     club|value_eur|preferred_foot|
+---------------+---+---------+---------+-----------+---------+---------+--------------+
| Z. Ibrahimović| 37|      195|       95|     Sweden|LA Galaxy| 14000000|         Right|
|           Pepe| 36|      188|       81|   Portugal| FC Porto|  6500000|         Right|
|      G. Buffon| 41|      192|       92|      Italy| Juventus|  2600000|         Right|
|       Casillas| 38|      185|       84|      Spain| FC Porto|  2600000|          Left|
|F. Quagliarella| 36|      180|       79|      Italy|Sampdoria|  9500000|         Right|
+---------------+---+---------+---------+-----------+---------+---------+--------------+
only showing top 5 rows



In [171]:
#buscando a los jugadores del Liverpool que tuviera mas de 30 años de edad
df_fifa.where((df_ejemplo.age > 30) & (df_ejemplo.club == 'Liverpool')).show(5)

+-----------+---+---------+---------+-----------+---------+---------+--------------+
| short_name|age|height_cm|weight_kg|nationality|     club|value_eur|preferred_foot|
+-----------+---+---------+---------+-----------+---------+---------+--------------+
|  J. Milner| 33|      175|       70|    England|Liverpool| 10500000|         Right|
| A. Lallana| 31|      178|       73|    England|Liverpool| 11000000|         Right|
|     Adrián| 32|      190|       80|      Spain|Liverpool|  4700000|         Right|
|A. Lonergan| 35|      192|       87|    England|Liverpool|   150000|          Left|
+-----------+---+---------+---------+-----------+---------+---------+--------------+



#### Funciones de agregación

Supongamos que queremos saber el costo total en euros de un equipo, lograríamos dicho objetivo sumando el valor de los jugadores de cada equipo. Esto lo podemos lograr usando la funcion groupBy() en conjunto con algunas funciones de agregación de la siguiente manera

In [172]:
df_fifa.groupBy('Club').sum('value_eur').show(5)

+--------------------+--------------+
|                Club|sum(value_eur)|
+--------------------+--------------+
|       Côte d'Ivoire|             0|
|          Göztepe SK|      43065000|
|CD Everton de Viñ...|      18075000|
|     Shonan Bellmare|       1410000|
|        Salford City|       7905000|
+--------------------+--------------+
only showing top 5 rows



In [173]:
#si quisieramos saber cuales son los 5 equipos mas caros hariamos lo siguiente
df_fifa.groupBy('club') \
  .sum('value_eur') \
  .orderBy('sum(value_eur)', ascending = False) \
  .show(5)

+---------------+--------------+
|           club|sum(value_eur)|
+---------------+--------------+
|    Real Madrid|     897850000|
|   FC Barcelona|     869300000|
|Manchester City|     845745000|
|       Juventus|     735475000|
|      Liverpool|     693265000|
+---------------+--------------+
only showing top 5 rows



In [174]:
#Si quisieramos usar más de una función de agregación tendriamos que hacer lo siguiente
df_fifa.groupBy('club') \
  .agg(sum('value_eur').alias('valor_total'), \
       mean('age').alias('edad_promedio'))\
  .orderBy('valor_total', ascending = False) \
  .show(5)

+---------------+-----------+------------------+
|           club|valor_total|     edad_promedio|
+---------------+-----------+------------------+
|    Real Madrid|  897850000| 24.90909090909091|
|   FC Barcelona|  869300000|24.060606060606062|
|Manchester City|  845745000|24.333333333333332|
|       Juventus|  735475000|              27.0|
|      Liverpool|  693265000|24.484848484848484|
+---------------+-----------+------------------+
only showing top 5 rows



#### Crear columnas

In [175]:
#Podemos crear columnas a partir de los valores de otras, por ejemplo, podríamos querer obtener el valor de los jugadores en pesos

In [176]:
df_fifa = df_fifa.withColumn('valor_pesos', df_fifa.value_eur * 20)

### Consultas en SQL

De manera general, podemos observar que existe cierta similaridad entre las funciones de los DataFrames de PySpark con algunas sentencias de SQL. Este paralelismo llega a tal grado que podemos reemplazar la concatenación de varias funciones con una simple consulta SQL.

In [177]:
#Por ejemplo, si quisieramos consultar de nuevo cuales son los 5 equipos más caros, solo que esta vez haciendo una consulta SQL, haríamos los siguiente

# Le dice a spark que guarde este DataFrame como una tabla SQL
df_fifa.registerTempTable('FIFA20')

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [178]:
consulta = '''
  SELECT club, SUM(value_eur) AS valor_total
  FROM FIFA20
  GROUP BY club
  ORDER BY valor_total DESC
  '''
spark.sql(consulta).show(5)

+---------------+-----------+
|           club|valor_total|
+---------------+-----------+
|    Real Madrid|  897850000|
|   FC Barcelona|  869300000|
|Manchester City|  845745000|
|       Juventus|  735475000|
|      Liverpool|  693265000|
+---------------+-----------+
only showing top 5 rows



**Ejercicio 1.** Visualizar los 30 jugadores más altos (mostrando su altura en pies ordenado de mayor a menor) y con un peso mayor a 70 kg.


In [179]:
consulta = '''
  SELECT short_name, height_cm / 30.48 as height_pies
  FROM FIFA20
  WHERE weight_kg > 70
  ORDER BY height_cm DESC
  '''
spark.sql(consulta).show(30)

+-------------------+-----------+
|         short_name|height_pies|
+-------------------+-----------+
|            T. Holý|   6.725722|
|           M. Casey|   6.660105|
|      C. Pantilimon|   6.660105|
|          L. Traoré|   6.660105|
|              A. Ba|   6.660105|
|         A. Chapman|   6.660105|
|           M. Qvist|   6.660105|
|          S. Brolin|   6.627297|
|V. Milinković-Savić|   6.627297|
|           D. Thiam|   6.627297|
|        K. Scherpen|   6.627297|
|         R. Laidlaw|   6.594488|
|         L. Kalinić|   6.594488|
|           K. Athiu|   6.594488|
|         F. Forster|   6.594488|
|          K. Letica|   6.594488|
|         P. Onuachu|   6.594488|
|         H. Veerman|   6.594488|
|         A. Vukotić|   6.594488|
|         A. Sjöberg|   6.594488|
|        S. Makienok|   6.594488|
|        E. Johansen|   6.594488|
|         H. Souttar|   6.561680|
|            N. Pope|   6.561680|
|       J. Karlstrøm|   6.561680|
|         A. Koșelev|   6.561680|
|       J. Bra

**Ejercicio 2.** Visualizar los 5 jugadores con valor en pesos más bajo que sean de nacionalidad mexicana, ordenado de menor a mayor.

In [180]:
consulta = '''
  SELECT short_name, valor_pesos
  FROM FIFA20
  WHERE nationality = 'Mexico'
  ORDER BY valor_pesos
  LIMIT 5
  '''
spark.sql(consulta).show()


+------------+-----------+
|  short_name|valor_pesos|
+------------+-----------+
|   H. Moreno|          0|
|R. Hernández|     600000|
|Y. Gutiérrez|     800000|
|   J. García|     800000|
|J. Rodríguez|     800000|
+------------+-----------+



**Ejercicio 3.** Visualizar los 10 jugadores con el peso más bajo de un club de su elección, ordenado de menor a mayor.


In [183]:
consulta = '''
  SELECT short_name, weight_kg
  FROM FIFA20
  WHERE club = "Liverpool"
  ORDER BY weight_kg ASC
  LIMIT 10
  '''
spark.sql(consulta).show(10)

+----------------+---------+
|      short_name|weight_kg|
+----------------+---------+
|        A. Lewis|       63|
|        N. Keïta|       64|
|    A. Robertson|       64|
|         R. Kent|       65|
|Pedro Chirivella|       66|
|      P. Glatzel|       67|
|        N. Clyne|       67|
|         H. Kane|       67|
|       K. Hoever|       67|
|        C. Jones|       68|
+----------------+---------+




**Ejercicio 4.** Visualizar los 20 jugadores de mayor edad de nacionalidad argentina, ordenados de mayor a menor con respecto a su peso.

In [185]:
consulta = '''
SELECT short_name, age, weight_kg, nationality
FROM FIFA20
WHERE nationality = 'Argentina'
ORDER BY age DESC, weight_kg DESC
'''
spark.sql(consulta).show(20)

+--------------+---+---------+-----------+
|    short_name|age|weight_kg|nationality|
+--------------+---+---------+-----------+
|      C. Muñoz| 42|       73|  Argentina|
|  C. Lucchetti| 41|       69|  Argentina|
|    M. Caranta| 40|       77|  Argentina|
|     F. Cubero| 40|       72|  Argentina|
| H. Campagnaro| 39|       85|  Argentina|
|    S. Torrico| 39|       84|  Argentina|
|C. Campestrini| 39|       82|  Argentina|
|       J. Sand| 38|       82|  Argentina|
|  M. Rodríguez| 38|       79|  Argentina|
|   J. Gandolfi| 38|       78|  Argentina|
|      L. Licht| 38|       72|  Argentina|
|    E. Bologna| 37|       87|  Argentina|
|   L. Pocrnjic| 37|       86|  Argentina|
|     N. Ibáñez| 37|       85|  Argentina|
|  F. Coloccini| 37|       83|  Argentina|
|    L. Giovini| 37|       82|  Argentina|
|  W. Caballero| 37|       81|  Argentina|
| F. Mancinelli| 37|       80|  Argentina|
|        G. Lux| 37|       77|  Argentina|
|    I. Damonte| 37|       72|  Argentina|
+----------